In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import file_methods as fm
import input_file as ipf
import generate_inputs as gi
import keyword_block
import sys
import subprocess
import pickle
import pandas as pd
import labels as lbls

In [2]:
# Import a template CT input file.
template = gi.import_template('CarbSulfModel/CSColumnShortSR.in')

*** Importing template file ***
The keyword "ION_EXCHANGE" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "SURFACE_COMPLEXATION" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
BaseException: This is normally due to a commented line in the input file. If it is not, something has gone really wrong!
The keyword "PEST" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "ISOTOPES" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.


In [263]:
dictionary={}
dictionary['primary_species']=dict.fromkeys(template.keyword_blocks['PRIMARY_SPECIES'].contents.keys())
dictionary['secondary_species']=dict.fromkeys(template.keyword_blocks['SECONDARY_SPECIES'].contents.keys())

del dictionary['primary_species']['PRIMARY_SPECIES']
del dictionary['secondary_species']['SECONDARY_SPECIES']



#Swap CO2 and H2S because of the base swap
dictionary['secondary_species']['CO2(aq)']=None
dictionary['secondary_species']['C13O2(aq)']=None
dictionary['secondary_species']['H2S(aq)']=None
dictionary['secondary_species']['H2S34(aq)']=None
dictionary['secondary_species'].pop('HCO3-')
dictionary['secondary_species'].pop('HC13O3-')
dictionary['secondary_species'].pop('HS-')
dictionary['secondary_species'].pop('HS34-')

dictionary['primary_species']['HCO3-']=None
dictionary['primary_species']['HC13O3-']=None
dictionary['primary_species']['HS-']=None
dictionary['primary_species']['HS34-']=None
dictionary['primary_species'].pop('CO2(aq)')
dictionary['primary_species'].pop('C13O2(aq)')
dictionary['primary_species'].pop('H2S(aq)')
dictionary['primary_species'].pop('H2S34(aq)')


display(dictionary['secondary_species'])

output=pd.DataFrame(columns=dictionary['primary_species'],index=dictionary['secondary_species'])
output['H2O']=np.nan
output['LogK']=np.nan

{'HNO3(aq)': None,
 'Al(OH)2+': None,
 'Al(SO4)2-': None,
 'Al(S34O4)2-': None,
 'AlO2-': None,
 'AlOH++': None,
 'CO3--': None,
 'C13O3--': None,
 'CaCl+': None,
 'CaCl2(aq)': None,
 'CaOH+': None,
 'CaSO4(aq)': None,
 'CaS34O4(aq)': None,
 'Fe(OH)2(aq)': None,
 'Fe(OH)3-': None,
 'Fe(OH)4--': None,
 'FeCl+': None,
 'FeCl2(aq)': None,
 'FeCl4--': None,
 'FeOH+': None,
 'FeSO4(aq)': None,
 'FeS34O4(aq)': None,
 'OH-': None,
 'H2PO4-': None,
 'H3PO4(aq)': None,
 'HAlO2(aq)': None,
 'HSO4-': None,
 'HS34O4-': None,
 'HSiO3-': None,
 'KCl(aq)': None,
 'KHSO4(aq)': None,
 'KHS34O4(aq)': None,
 'KOH(aq)': None,
 'KSO4-': None,
 'KS34O4-': None,
 'NaAlO2(aq)': None,
 'NaCl(aq)': None,
 'NaHSiO3(aq)': None,
 'NaOH(aq)': None,
 'NaSO4-': None,
 'NaS34O4-': None,
 'NH4+': None,
 'CaHPO4(aq)': None,
 'CaPO4-': None,
 'FeCO3(aq)': None,
 'FeC13O3(aq)': None,
 'FeH2PO4+': None,
 'FeHPO4(aq)': None,
 'FePO4-': None,
 'KHPO4-': None,
 'PO4---': None,
 'S--': None,
 'S34--': None,
 'BO2-': None,
 'Mg

In [264]:

my_cols=range(0,32)
df=pd.read_csv('tmp/reducedDatabaseCarbonIsotopes.dbs',delim_whitespace=True,header=None,names=my_cols,quotechar="'")
df = df.fillna(value=np.nan)
#display(df.loc[35,:])
#display(test[test.last_valid_index()])
#Find index which is end of primary species
A=df[0].str.contains("End of primary")
startsec=1+np.where(A)[0][0]
print(startsec)
#Find index which is end of secondary species
B=df[0].str.contains("End of secondary")
endsec=np.where(B)[0][0]
print(endsec)



36
211


In [265]:
sec_species=df.iloc[startsec:endsec][:]
display(sec_species)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
36,Acetic_acid(aq),3,-2.0000,O2(g),2.0000,H+,2.0000,HCO3-,150.4629,136.1956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,S2--,3,-1.0000,H2O,0.5000,O2(g),2.0000,HS-,-35.3940,-31.8181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,S2O3--,4,-2.0000,O2(g),-1.0000,H2O,2.0000,H+,2.0000,SO4--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,BH4-,4,-2.0000,O2(g),-1.0000,H+,1.0000,B(OH)3(aq),1.0000,H2O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,CN-,4,-2.0000,H2O,-0.5000,O2(g),1.0000,HCO3-,1.0000,NH3(aq),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,S--,2,-1.0000,H+,1.0000,HS-,13.7100,12.9351,12.0082,11.1018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,S34--,2,-1.0000,H+,1.0000,HS34-,13.7100,12.9351,12.0082,11.1018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,S2O5--,2,1.0000,O2(g),1.0000,S2O3--,-52.0852,-46.9857,-41.1293,-35.7772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,SO2(aq),3,-1.0000,H2O,1.0000,SO3--,2.0000,H+,-8.7829,-9.0656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
print(output)

            H2(aq)   H+ Al+++ NH3(aq) Mg++ Ca++ Fe++ Fe+++  Cl-   K+  ...  \
HNO3(aq)       NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
Al(OH)2+       NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
Al(SO4)2-      NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
Al(S34O4)2-    NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
AlO2-          NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
AlOH++         NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
CO3--          NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
C13O3--        NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
CaCl+          NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
CaCl2(aq)      NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
CaOH+          NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   
CaSO4(aq)      NaN  NaN   NaN     NaN  NaN  NaN  NaN   NaN  NaN  NaN  ...   

In [270]:

for row in sec_species.iterrows():
    #print(row[1])
    string=str(row[1][0])
    #print(string)
    nospecies=int(row[1][1])
    #print(nospecies)
    specname=row[1][np.linspace(3,3+((nospecies-1)*2),nospecies)].tolist()
    #print(specname)
    specstoic=row[1][np.linspace(2,2+((nospecies-1)*2),nospecies)].tolist()
    #print(specstoic)
    temppoints=row[1][range(2+(nospecies*2),10+(nospecies*2))].tolist()
    #print(temppoints)
    #print(rowoutput[0])
    if sum(output.index==string)==1:
        for x in range(0,nospecies):
            output.loc[string,specname[x]]=specstoic[x]
        output.loc[string,'LogK']=float(temppoints[0])+((float(temppoints[1])-float(temppoints[0]))*(8/25))


        
print(output)    

output.to_excel('LogK.xlsx')



            H2(aq)       H+   Al+++ NH3(aq)    Mg++    Ca++    Fe++ Fe+++  \
HNO3(aq)       NaN   1.0000     NaN     NaN     NaN     NaN     NaN   NaN   
Al(OH)2+       NaN  -2.0000  1.0000     NaN     NaN     NaN     NaN   NaN   
Al(SO4)2-      NaN      NaN  1.0000     NaN     NaN     NaN     NaN   NaN   
Al(S34O4)2-    NaN      NaN  1.0000     NaN     NaN     NaN     NaN   NaN   
AlO2-          NaN  -4.0000  1.0000     NaN     NaN     NaN     NaN   NaN   
AlOH++         NaN  -1.0000  1.0000     NaN     NaN     NaN     NaN   NaN   
CO3--          NaN  -1.0000     NaN     NaN     NaN     NaN     NaN   NaN   
C13O3--        NaN  -1.0000     NaN     NaN     NaN     NaN     NaN   NaN   
CaCl+          NaN      NaN     NaN     NaN     NaN  1.0000     NaN   NaN   
CaCl2(aq)      NaN      NaN     NaN     NaN     NaN  1.0000     NaN   NaN   
CaOH+          NaN  -1.0000     NaN     NaN     NaN  1.0000     NaN   NaN   
CaSO4(aq)      NaN      NaN     NaN     NaN     NaN  1.0000     NaN   NaN   